In [1]:
#use traitlets and widgets to display the image in Jupyter Notebook
import traitlets
from traitlets.config.configurable import SingletonConfigurable

#use opencv to covert the depth image to RGB image for displaying purpose
import cv2
import numpy as np

from RobotClass import Robot
from CameraClass import Camera

#using realsense to capture the color and depth image
import pyrealsense2 as rs

#multi-threading is used to capture the image in real time performance
import threading

class Camera(SingletonConfigurable):
    
    #this changing of this value will be captured by traitlets
    value = traitlets.Any()
    
    def __init__(self):
        super(Camera, self).__init__()
        
        #configure the color and depth sensor
        self.pipeline = rs.pipeline()
        self.configuration = rs.config()  
        
        #set resolution for the color camera
        self.color_width = 640
        self.color_height = 480
        self.color_fps = 30
        self.configuration.enable_stream(rs.stream.color, self.color_width, self.color_height, rs.format.bgr8, self.color_fps)

        #set resolution for the depth camera
        self.depth_width = 640
        self.depth_height = 480
        self.depth_fps = 30
        self.configuration.enable_stream(rs.stream.depth, self.depth_width, self.depth_height, rs.format.z16, self.depth_fps)

        #flag to control the thread
        self.thread_runnning_flag = False
        
        #start the RGBD sensor
        self.pipeline.start(self.configuration)
        self.pipeline_started = True
        frames = self.pipeline.wait_for_frames()

        #start capture the first color image
        color_frame = frames.get_color_frame()   
        image = np.asanyarray(color_frame.get_data())
        self.value = image

        #start capture the first depth image
        depth_frame = frames.get_depth_frame()           
        depth_image = np.asanyarray(depth_frame.get_data())
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        self.depth_value = depth_colormap   

    def _capture_frames(self):
        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            frames = self.pipeline.wait_for_frames() #receive data from RGBD sensor
            
            color_frame = frames.get_color_frame() #get the color image
            image = np.asanyarray(color_frame.get_data()) #convert color image to numpy array
            self.value = image #assign the numpy array image to the color_value variable 

            depth_frame = frames.get_depth_frame() #get the depth image           
            depth_image = np.asanyarray(depth_frame.get_data()) #convert depth data to numpy array
            #conver depth data to BGR image for displaying purpose
            depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
            self.depth_value = depth_colormap #assign the color BGR image to the depth value
    
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread       

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera.instance()
camera.start() # start capturing the data

In [2]:
import cv2
import numpy as np
import time
from drivers import PCA9685, Motor
from traitlets.config.configurable import SingletonConfigurable
import traitlets

def line_detection(image, lower_color, upper_color):

    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower_color, upper_color)
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

 

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        moment = cv2.moments(largest_contour)

        if moment["m00"] != 0:
            cx = int(moment["m10"] / moment["m00"])
            cy = int(moment["m01"] / moment["m00"])
            return (True, cx, cy)

    return (False, None, None)

 

def control_robot(robot, x, y, image_width, has_checked, t):
    
    if x is not None and y is not None:
        t = 0.1
        center = image_width // 2
        error = x - center

        if abs(error) < 50:
            robot.forward(0.6)
        elif error < 0:
            robot.left(0.4)
        else:
            robot.right(0.4)
    elif t >= 0.3:
        robot.backward(0.2)
        t = 0.1
        time.sleep(0.2)
        has_checked = False
    elif not has_checked:
        robot.left(0.6)
        time.sleep(t)
        has_checked = True
        t += 0.05
    elif has_checked:
        robot.right(0.6)
        time.sleep(t)
        has_checked = False
        t += 0.05
        
    return has_checked, t

 

def main():
    
    has_checked = False
    t = 0.1

    # Initialize the robot and camera

    robot = Robot()

    # Define the color range for the line (e.g., yellow or blue)

    lower_color = np.array([20, 100, 100])  # Lower bound of the yellow color in HSV
    upper_color = np.array([30, 255, 255])  # Upper bound of the yellow color in HSV

    try:
        while True:

            # Get the current frame from the camera
            frame = camera.value

            # Detect the line's position in the frame
            found, x, y = line_detection(frame, lower_color, upper_color)
            
            if found and has_checked:
                has_checked = not has_checked
            elif not found and has_checked:
                has_checked = not has_checked

            # Control the robot based on the line's position
            has_checked = control_robot(robot, x, y, frame.shape[1], has_checked, t)

            # Sleep for a while to reduce CPU usage
            time.sleep(0.1)

    except KeyboardInterrupt:
        print("Interrupted")

    finally:
        robot.stop()


if __name__ == "__main__":
    main()

Interrupted


In [ ]:
# import cv2
# import numpy as np
# import time
# from drivers import PCA9685, Motor
# from traitlets.config.configurable import SingletonConfigurable
# import traitlets

# def line_detection(image, lower_color, upper_color):

#     hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
#     mask = cv2.inRange(hsv, lower_color, upper_color)
#     contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

 

#     if len(contours) > 0:
#         largest_contour = max(contours, key=cv2.contourArea)
#         moment = cv2.moments(largest_contour)

#         if moment["m00"] != 0:
#             cx = int(moment["m10"] / moment["m00"])
#             cy = int(moment["m01"] / moment["m00"])
#             return (True, cx, cy)

#     return (False, None, None)

 

# def control_robot(robot, x, y, image_width, last_turn, t):
    
#     if x is not None and y is not None:
#         t = 0.1
#         center = image_width // 2
#         error = x - center

#         if abs(error) < 50:
#             robot.forward(0.6)
#         elif error < 0:
#             robot.left(0.4)
#             last_turn = "left"
#         else:
#             robot.right(0.4)
#             last_turn = "right"
#     elif t >= 0.3:
#         robot.backward(0.2)
#         t = 0.1
#         time.sleep(0.2)
#         if last_turn == 'right':
#             last_turn = "left"
#         else:
#             last_turn = "right"
#     elif last_turn == "right":
#         robot.left(0.6)
#         time.sleep(t)
#         last_turn = "left"
#         t += 0.05
#     elif last_turn == "left":
#         robot.right(0.6)
#         time.sleep(t)
#         last_turn = "right"
#         t += 0.05
        
#     return last_turn, t

 

# def main():
    
#     last_turn = None

#     t = 0.1

#     # Initialize the robot and camera

#     robot = Robot()

#     # Define the color range for the line (e.g., yellow or blue)

#     lower_color = np.array([20, 100, 100])  # Lower bound of the yellow color in HSV
#     upper_color = np.array([30, 255, 255])  # Upper bound of the yellow color in HSV

#     try:
#         while True:

#             # Get the current frame from the camera
#             frame = camera.value

#             # Detect the line's position in the frame
#             found, x, y = line_detection(frame, lower_color, upper_color)
            
# #             if found and has_checked:
# #                 has_checked = not has_checked
# #             elif not found and has_checked:
# #                 has_checked = not has_checked

#             # Control the robot based on the line's position
#             last_turn = control_robot(robot, x, y, frame.shape[1], last_turn, t)

#             # Sleep for a while to reduce CPU usage
#             time.sleep(0.05)

#     except KeyboardInterrupt:
#         print("Interrupted")

#     finally:
#         robot.stop()


# if __name__ == "__main__":
#     main()

In [3]:
robot = Robot()
robot.stop()